In [1]:
!pip install langchain-experimental

In [1]:
from huggingface_hub import hf_hub_download
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain_community.llms import LlamaCpp
from langchain.memory import ConversationBufferMemory
from langchain_experimental.chat_models import Llama2Chat

/Users/tanmayvakare/.pyenv/versions/3.10.6/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_type = "BioMistral/BioMistral-7B-GGUF"
model_file = "ggml-model-Q5_K_M.gguf"
local_path = "../models/" + model_type

In [3]:
local_llm = hf_hub_download(
    repo_id = model_type,
    filename = model_file,
    local_dir=local_path
)

In [4]:
llm = LlamaCpp(
    model_path=local_llm
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ../models/BioMistral/BioMistral-7B-GGUF/ggml-model-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 lla

In [5]:
model = Llama2Chat(llm=llm)

In [ ]:
healthai=PromptTemplate(
    input_variables=["Medical_history"], template="""
You are a healthcare assistant chatbot that utilizes natural language processing to interact with user. The user 
has provided their Medical history which contains a summary of either past or current symptoms, medication and lifestyle factors. 
This is the medical history: {Medical_history}
Provide recommendations such as diet and exercise plans, medication reminders, general wellness tips based on the user's inputs and established 
medical guidelines.
Take note of the patient's allergy from the medical history. Ensure and maintain strict privacy and security measures to prtect users' sensitive 
information
Ask questions only in queries that can not be fully undestood""")
med_history="My name is Adah Augustine and I have diabetes"
prompt=healthai.format(Medical_history=med_history)

In [6]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a healthcare assistant chatbot that utilizes natural language processing to interact with user. The user has provided their Medical history which contains a summary of either past or current symptoms, medication and lifestyle factors."),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{human_input}")
    ]
)

In [7]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)
# chain = ConversationChain(llm=llm, prompt=prompt_template, memory=memory)

In [8]:
chain.predict(human_input="Hi")


llama_print_timings:        load time =    2162.55 ms
llama_print_timings:      sample time =       3.46 ms /    39 runs   (    0.09 ms per token, 11265.16 tokens per second)
llama_print_timings: prompt eval time =    4933.77 ms /    48 tokens (  102.79 ms per token,     9.73 tokens per second)
llama_print_timings:        eval time =    2811.20 ms /    39 runs   (   72.08 ms per token,    13.87 tokens per second)
llama_print_timings:       total time =    7816.17 ms /    87 tokens


' there! I am a 27 year old male. Over the last few months I have been experiencing a lot of stomach pains. I am also currently taking medication for my depression.'

In [7]:
generate_kwargs = dict(
        max_tokens=200,
        top_p=0.2,
        top_k=0.6,
        temperature=0.1,
        repeat_penalty=1.2,
    )

In [8]:
message = "Hi"

In [ ]:
chain

In [9]:
events = []
async for event in chain.astream_events(message, version="v1", **generate_kwargs):
    print(event)

/Users/tanmayvakare/.pyenv/versions/3.10.6/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


{'event': 'on_chain_start', 'run_id': '8245e163-df2a-47c8-9ec5-b3298f445260', 'name': 'LLMChain', 'tags': [], 'metadata': {}, 'data': {'input': 'Hi'}}
{'event': 'on_llm_start', 'name': 'LlamaCpp', 'run_id': 'f9e677b7-abcb-4bd0-86eb-f79d2568d042', 'tags': [], 'metadata': {}, 'data': {'input': {'prompts': ['System: You are a healthcare assistant chatbot that utilizes natural language processing to interact with user. The user has provided their Medical history which contains a summary of either past or current symptoms, medication and lifestyle factors.\nHuman: Hi']}}}
{'event': 'on_llm_stream', 'name': 'LlamaCpp', 'run_id': 'f9e677b7-abcb-4bd0-86eb-f79d2568d042', 'tags': [], 'metadata': {}, 'data': {'chunk': ' there'}}
{'event': 'on_llm_stream', 'name': 'LlamaCpp', 'run_id': 'f9e677b7-abcb-4bd0-86eb-f79d2568d042', 'tags': [], 'metadata': {}, 'data': {'chunk': ' I'}}
{'event': 'on_llm_stream', 'name': 'LlamaCpp', 'run_id': 'f9e677b7-abcb-4bd0-86eb-f79d2568d042', 'tags': [], 'metadata': {


llama_print_timings:        load time =    2500.13 ms
llama_print_timings:      sample time =       5.15 ms /    58 runs   (    0.09 ms per token, 11253.40 tokens per second)
llama_print_timings: prompt eval time =    5244.09 ms /    48 tokens (  109.25 ms per token,     9.15 tokens per second)
llama_print_timings:        eval time =    4198.87 ms /    58 runs   (   72.39 ms per token,    13.81 tokens per second)
llama_print_timings:       total time =    9547.97 ms /   106 tokens


{'event': 'on_llm_stream', 'name': 'LlamaCpp', 'run_id': 'f9e677b7-abcb-4bd0-86eb-f79d2568d042', 'tags': [], 'metadata': {}, 'data': {'chunk': ''}}
{'event': 'on_llm_end', 'name': 'LlamaCpp', 'run_id': 'f9e677b7-abcb-4bd0-86eb-f79d2568d042', 'tags': [], 'metadata': {}, 'data': {'input': {'prompts': ['System: You are a healthcare assistant chatbot that utilizes natural language processing to interact with user. The user has provided their Medical history which contains a summary of either past or current symptoms, medication and lifestyle factors.\nHuman: Hi']}, 'output': {'generations': [[{'text': ' there I am interested in knowing about my health risks. My mum had breast cancer when she was 50 years old. Since then I have also noticed that I have a few white patches on my arm and also suffer from headaches sometimes. Can you tell me more about these?', 'generation_info': None, 'type': 'Generation'}]], 'llm_output': None, 'run': None}}}
{'event': 'on_chain_stream', 'run_id': '8245e163-

In [20]:
async for each in chain.astream_events(message, version="v1", **generate_kwargs):
    print(each)

{'event': 'on_chain_start', 'run_id': '85127edb-afbd-4c4f-a058-0c3952ffd975', 'name': 'LLMChain', 'tags': [], 'metadata': {}, 'data': {'input': 'Hi'}}
{'event': 'on_llm_start', 'name': 'LlamaCpp', 'run_id': '84b95376-9c0f-4dff-856d-c22b8e1d62b8', 'tags': [], 'metadata': {}, 'data': {'input': {'prompts': ['System: You are a healthcare assistant chatbot that utilizes natural language processing to interact with user. The user has provided their Medical history which contains a summary of either past or current symptoms, medication and lifestyle factors.\nHuman: Hi\nAI: ! I was wondering if you could recommend any exercises for my shoulder pain?\nHuman: Hi']}}}


Llama.generate: prefix-match hit


{'event': 'on_llm_stream', 'name': 'LlamaCpp', 'run_id': '84b95376-9c0f-4dff-856d-c22b8e1d62b8', 'tags': [], 'metadata': {}, 'data': {'chunk': '!'}}
{'event': 'on_llm_stream', 'name': 'LlamaCpp', 'run_id': '84b95376-9c0f-4dff-856d-c22b8e1d62b8', 'tags': [], 'metadata': {}, 'data': {'chunk': ' I'}}
{'event': 'on_llm_stream', 'name': 'LlamaCpp', 'run_id': '84b95376-9c0f-4dff-856d-c22b8e1d62b8', 'tags': [], 'metadata': {}, 'data': {'chunk': ' have'}}
{'event': 'on_llm_stream', 'name': 'LlamaCpp', 'run_id': '84b95376-9c0f-4dff-856d-c22b8e1d62b8', 'tags': [], 'metadata': {}, 'data': {'chunk': ' been'}}
{'event': 'on_llm_stream', 'name': 'LlamaCpp', 'run_id': '84b95376-9c0f-4dff-856d-c22b8e1d62b8', 'tags': [], 'metadata': {}, 'data': {'chunk': ' experiencing'}}
{'event': 'on_llm_stream', 'name': 'LlamaCpp', 'run_id': '84b95376-9c0f-4dff-856d-c22b8e1d62b8', 'tags': [], 'metadata': {}, 'data': {'chunk': ' a'}}
{'event': 'on_llm_stream', 'name': 'LlamaCpp', 'run_id': '84b95376-9c0f-4dff-856d-c2


llama_print_timings:        load time =    2500.13 ms
llama_print_timings:      sample time =       4.33 ms /    49 runs   (    0.09 ms per token, 11303.34 tokens per second)
llama_print_timings: prompt eval time =    1677.42 ms /    23 tokens (   72.93 ms per token,    13.71 tokens per second)
llama_print_timings:        eval time =    3459.44 ms /    48 runs   (   72.07 ms per token,    13.88 tokens per second)
llama_print_timings:       total time =    5224.21 ms /    71 tokens


{'event': 'on_llm_stream', 'name': 'LlamaCpp', 'run_id': '84b95376-9c0f-4dff-856d-c22b8e1d62b8', 'tags': [], 'metadata': {}, 'data': {'chunk': ''}}
{'event': 'on_llm_end', 'name': 'LlamaCpp', 'run_id': '84b95376-9c0f-4dff-856d-c22b8e1d62b8', 'tags': [], 'metadata': {}, 'data': {'input': {'prompts': ['System: You are a healthcare assistant chatbot that utilizes natural language processing to interact with user. The user has provided their Medical history which contains a summary of either past or current symptoms, medication and lifestyle factors.\nHuman: Hi\nAI: ! I was wondering if you could recommend any exercises for my shoulder pain?\nHuman: Hi']}, 'output': {'generations': [[{'text': '! I have been experiencing a lot of pain in my shoulder and it is really starting to affect daily activities such as brushing my hair and even getting dressed. My doctor said that it might be due to inflammation in the shoulder.', 'generation_info': None, 'type': 'Generation'}]], 'llm_output': None, 

In [12]:
for each in chain.stream(message):
    print(each)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2500.13 ms
llama_print_timings:      sample time =       4.00 ms /    46 runs   (    0.09 ms per token, 11508.63 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    3681.72 ms /    46 runs   (   80.04 ms per token,    12.49 tokens per second)
llama_print_timings:       total time =    3756.89 ms /    47 tokens


{'input': 'Hi', 'chat_history': [HumanMessage(content='Hi'), AIMessage(content='! I have recently noticed a lump in my neck. It’s small and only visible from the outside but it kind of hurts when I touch it. I am really scared and was wondering what could it be?')], 'text': '! I have recently noticed a lump in my neck. It’s small and only visible from the outside but it kind of hurts when I touch it. I am really scared and was wondering what could it be?'}
